# Import

In [10]:
from xgboost import XGBClassifier
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer, precision_score, recall_score, roc_auc_score, roc_curve, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import datetime
import warnings
# 유니코드 깨짐현상 해결
mpl.rcParams['axes.unicode_minus'] = False

# 나눔고딕 폰트 적용
plt.rcParams["font.family"] = 'AppleGothic'

# 경고 무시
warnings.filterwarnings('ignore')
%matplotlib inline



# File read & Value sorting

In [78]:


a = pd.read_csv("C:/Users/wjddb/LotteMembers/아카이브/all.csv")

#거래날짜
a['거래날짜'] = pd.to_datetime(a['거래날짜'])
a['거래년도'] = a['거래날짜'].dt.year
a['거래월'] = a['거래날짜'].dt.month
a['거래일'] = a['거래날짜'].dt.day
a['거래분기']= a['거래날짜'].dt.quarter
#카테고리화
#a['성별'] = a['성별'].astype('category')
#a['나이대'] = a['나이대'].astype('category')
a['성별나이'] = a['성별나이'].astype('category')
a['지역'] = a['지역'].astype('category')
a['온오프'] = a['온오프'].astype('category')
a['제휴사'] = a['제휴사'].astype('category')
a['상품대분류'] = a['상품대분류'].astype('category')
a['회원유무'] = a['회원유무'].astype('category')
a['상품클래스'] = a['상품클래스'].astype('category')
a['거래요일'] = a['거래요일'].astype('category')
#a['RFMClass'] = a['RFMClass'].astype('category')
columns_no_need = ['고객번호','영수증번호','거래날짜','상품이름','상품소분류','R_Quantile','F_Quantile','M_Quantile','RFMClass']
#필요없는 컬럼 정리
a = a.drop(columns_no_need,axis=1)

# 데이터가 너무 많아서 20%의 랜덤 데이터 만을 사용.
a = a.sample(frac=0.2)
# XGBoost 에 맞추기 위하여 모델의 DUMMIFY 진행.
a = pd.get_dummies(a,prefix_sep='_',drop_first=True)

**CHECK VALUES**

In [44]:
a.info

<bound method DataFrame.info of          거래시간     구매금액  구매수량  RFMClassScore  거래년도  거래월  거래일  거래분기  지역_Z02  \
1465166    17   4950.0   1.0             18  2021    1   10     1       0   
2603368    13   2120.0   1.0             16  2021    3   20     1       0   
3297906    18   1980.0   1.0              6  2021    7    7     3       0   
870816     16   2500.0   1.0             15  2021    5   30     2       0   
3049453    18   1000.0   1.0             18  2021    1   19     1       0   
...       ...      ...   ...            ...   ...  ...  ...   ...     ...   
3400509    18   5000.0   1.0             18  2021   11    4     4       0   
681562     11   3300.0   2.0             17  2021    3   20     1       0   
173891     20  24590.0   1.0             18  2021    2   21     1       0   
697554     15   5500.0   1.0             18  2021    8   13     3       0   
3359961    16  13900.0   1.0             16  2021    5    3     2       0   

         지역_Z03  ...  상품클래스_유아  상품클래스_인테리어 

# Creating Model

In [79]:
from sklearn.preprocessing import LabelEncoder
X = a.drop(columns=['RFMClassScore'])
y = a['RFMClassScore']

X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.2,random_state=2)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
# 모델 선언
xgb = XGBClassifier(n_estimators=100, learning_rate=0.2, max_depth=4, random_state = 323)
import time

# 모델 훈련
start_time = time.time()
xgb.fit(X_train, y_train,verbose=False)
print('Fit time : ', time.time() - start_time)

Fit time :  1178.5655765533447


**PREDICT**

In [80]:
predictions = xgb.predict(X_test)
r_sq = xgb.score(X_train, y_train)
print(r_sq)
print(explained_variance_score(predictions,y_test))

0.27301307898008526
-3.826151859304667


**ACCURACY**

In [81]:
#Measuring accuracy on Testing Data
print('Accuracy',100 - (np.mean(np.abs((y_test - predictions) / y_test)) * 100))

Accuracy 77.61587508885808


In [83]:
a = pd.DataFrame(predictions)

a = pd.concat([a,y_test.reset_index(drop=True)],axis=1)
a

,0,RFMClassScore
0,13,16
1,15,18
2,14,18
3,15,12
4,15,12
...,...,...
185197,14,15
185198,14,18
185199,15,12
185200,13,6


# FILE OUT

In [77]:
a.to_csv("C:/Users/wjddb/LotteMembers/아카이브/predictions2.csv")